In [ ]:
from langchain_community.document_loaders import TextLoader
loader=TextLoader("youtube_transcript.txt")
text_doc=loader.load()
text_doc

In [ ]:
from langchain_community.document_loaders import WebBaseLoader
from bs4 import SoupStrainer

loader = WebBaseLoader(
    web_path="https://www.aljazeera.com/news/liveblog/2025/6/20/live-iran-israel-continue-missile-fire-irans-fm-to-meet-eu-counterparts",
    bs_kwargs=dict(parse_only=SoupStrainer(class_="article-header"))
)

k = loader.load()
k

In [ ]:
from langchain_community.document_loaders import PyPDFLoader
loader=PyPDFLoader("report (1).pdf")
pdf_doc=loader.load()
pdf_doc

In [12]:
from langchain.text_splitter import RecursiveCharacterTextSplitter
split=RecursiveCharacterTextSplitter(chunk_size=1000,chunk_overlap=200)
text=split.split_documents(pdf_doc)

In [ ]:
from langchain_community.vectorstores import FAISS
from langchain_community.embeddings import OllamaEmbeddings
embed=OllamaEmbeddings(model="llama3.2:latest")
db_f=FAISS.from_documents(text[0:20], embedding=embed)


In [35]:
result=db_f.similarity_search("what is main topic")

In [15]:
from langchain_core.prompts import ChatPromptTemplate
template=ChatPromptTemplate.from_template(
    """you are an intelligent AI and u will answer the questions based on the context provided
     and u are in a competetiton with a similar AI and if the user finds the other ai answer more useful it will stop using u
       be very precise and informative in your answer and maintain a high level of accuracy and be a friendly talkative AI.
       <context>
       {context}
       </context>
       question:{input}"""
)

In [ ]:
from langchain.chains.combine_documents import create_stuff_documents_chain
from langchain_groq import ChatGroq
import os
os.environ["GROQ_API_KEY"]="your api key"
model1=ChatGroq(model="meta-llama/llama-4-scout-17b-16e-instruct",temperature=0.2)
chain1=create_stuff_documents_chain(model1,template)
response = chain1.invoke({
    "input": "what is the maintopic of the report?",
    "context": result 
})
print(response)

In [ ]:
retrieve=db_f.as_retriever
retrieve

In [ ]:
from langchain.chains import create_retrieval_chain
retriever = db_f.as_retriever()
chain2 = create_retrieval_chain(retriever, chain1)

# Run the retrieval chain with a query
response = chain2.invoke({"input": "what is the maintopic of the report?"})
print(response)